In [27]:
import pandas as pd
from tqdm import tqdm
import json

In [12]:
DIR_HOURLY = 'Z:/lab/lifebehavior/2020_jikoku_danjonenso.csv'
DIR_GENRE = 'Z:/lab/lifebehavior/nhk_genre.csv'

In [18]:
# データフレームを参照リストに基づいてソートするヘルパー関数
def sort_df_by_list(df, column, reference_list):
    return df.set_index(column).reindex(reference_list).reset_index()
# 性別と年齢範囲の変換関数
def create_key(label, day):
    # 曜日変換の辞書
    day_mapping = {
        "平日": "MONDAY",
        "土曜日": "SATURDAY",
        "日曜日": "SUNDAY"
    }
    # 性別を判断
    gender = "MALE" if label.startswith("男") else "FEMALE"

    # 年齢範囲を抽出
    if "１０代" in label:
        age_group = "10"
    elif "２０代" in label:
        age_group = "20"
    elif "３０代" in label:
        age_group = "30"
    elif "４０代" in label:
        age_group = "40"
    elif "５０代" in label:
        age_group = "50"
    elif "６０代" in label:
        age_group = "60"
    else:
        age_group = "70"  # "７０歳以上"の場合

    # 曜日を英語に変換
    english_day = day_mapping[day]

    # 最終的なキーを作成
    return f"{gender}_{age_group}-{english_day}"

# 独自定義のラベルを適用するためにまず辞書を作る
small = [
    "睡眠", "食事", "身のまわりの用事", "療養・静養", "仕事", "仕事のつきあい",
    "授業・学内の活動", "学校外の学習", "炊事・掃除・洗濯", "買い物",
    "子どもの世話", "家庭雑事", "通勤", "通学", "社会参加", "会話・交際",
    "スポーツ", "行楽・散策", "趣味・娯楽・教養(インターネット除く)",
    "趣味・娯楽・教養のインターネット(動画除く)", "インターネット動画",
    "テレビ", "録画番組・DVD", "ラジオ", "新聞", "雑誌・マンガ・本",
    "音楽", "休息", "その他", "不明"
]
label_master = pd.read_csv(DIR_GENRE)
label_dict = {}
for k,v in zip(label_master["小分類"],label_master["モデル用定義"]):
    label_dict[k]=v

# 行動種別の並び順管理
ACTIVITY_ORDERING = ['睡眠', '食事', '身のまわりの用事', '療養・静養', '仕事', '仕事のつきあい', '授業・学内の活動',
                     '学校外の学習', '炊事・掃除・洗濯', '買い物', '子どもの世話', '家庭雑事', '通勤', '通学', '社会参加',
                     '会話・交際', 'スポーツ', '行楽・散策', '趣味・娯楽・教養', 'マスメディア接触', '休息']

In [28]:
initialBehaviorProb = {}
for label in tqdm([
    '男１０代', '男２０代', '男３０代', '男４０代', '男５０代', '男６０代', '男７０歳以上',
    '女１０代', '女２０代', '女３０代', '女４０代', '女５０代', '女６０代', '女７０歳以上']):
    for day in ["平日", "土曜日", "日曜日"]:
        key = create_key(label,day)
        
        data_hourly = pd.read_csv(DIR_HOURLY)
        data_hourly = data_hourly.set_axis(["Day","Group","Activity","Time","Rate"],axis=1)
        # data_hourly["Time"] = pd.to_datetime(data_hourly["Time"], format="%H:%M")
        data_hourly = data_hourly.query("Time == '0:00'")
        data_hourly = data_hourly.query("Group == @label & Activity in @small & Day == @day")
        data_hourly["Activity"] = data_hourly["Activity"].map(label_dict) # 独自定義のラベル適用
        data_hourly_cleaned = data_hourly[["Activity","Time","Group","Rate"]].groupby(["Time","Activity","Group"],as_index=False).sum().copy()
        data_hourly_sorted = sort_df_by_list(data_hourly_cleaned,"Activity",ACTIVITY_ORDERING)
        prob_dic = dict(zip(data_hourly_sorted["Activity"],data_hourly_sorted["Rate"]/100))
        
        initialBehaviorProb[key] = prob_dic

# 生成された initialBehaviorProb を JSON ファイルに書き込む
output_file = "Y:\\生活行動モデル\\data\\initialBehaviorProbabilities.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(initialBehaviorProb, f, ensure_ascii=False, indent=4)

100%|██████████| 14/14 [00:06<00:00,  2.30it/s]


In [26]:
initialBehaviorProb

{'MALE_10-MONDAY': {'睡眠': 0.866,
  '食事': 0.005,
  '身のまわりの用事': 0.011000000000000001,
  '療養・静養': 0.0,
  '仕事': 0.0,
  '仕事のつきあい': 0.0,
  '授業・学内の活動': 0.0,
  '学校外の学習': 0.021,
  '炊事・掃除・洗濯': 0.005,
  '買い物': 0.0,
  '子どもの世話': 0.0,
  '家庭雑事': 0.0,
  '通勤': 0.0,
  '通学': 0.0,
  '社会参加': 0.0,
  '会話・交際': 0.0,
  'スポーツ': 0.0,
  '行楽・散策': 0.0,
  '趣味・娯楽・教養': 0.021,
  'マスメディア接触': 0.032,
  '休息': 0.011000000000000001},
 'MALE_10-SATURDAY': {'睡眠': 0.929,
  '食事': 0.0,
  '身のまわりの用事': 0.0,
  '療養・静養': 0.0,
  '仕事': 0.0,
  '仕事のつきあい': 0.0,
  '授業・学内の活動': 0.0,
  '学校外の学習': 0.012,
  '炊事・掃除・洗濯': 0.0,
  '買い物': 0.0,
  '子どもの世話': 0.0,
  '家庭雑事': 0.0,
  '通勤': 0.0,
  '通学': 0.0,
  '社会参加': 0.0,
  '会話・交際': 0.012,
  'スポーツ': 0.0,
  '行楽・散策': 0.0,
  '趣味・娯楽・教養': 0.012,
  'マスメディア接触': 0.012,
  '休息': 0.0},
 'MALE_10-SUNDAY': {'睡眠': 0.8540000000000001,
  '食事': 0.0,
  '身のまわりの用事': 0.0,
  '療養・静養': 0.0,
  '仕事': 0.0,
  '仕事のつきあい': 0.0,
  '授業・学内の活動': 0.0,
  '学校外の学習': 0.012,
  '炊事・掃除・洗濯': 0.0,
  '買い物': 0.0,
  '子どもの世話': 0.0,
  '家庭雑事': 0.0,
  '通勤': 0.0,